In [8]:
import numpy as np
import pandas as pd
from ztf_data import load_ztf_nearest_ast, make_ztf_easy_batch, load_ztf_easy_batch, make_ztf_near_elt, report_ztf_score
from asteroid_dataframe import calc_ast_data, spline_ast_vec_df, calc_ast_dir
from asteroid_data import orbital_element_batch
from astro_utils import deg2dist
from search_score_functions import score_mean_var
from tqdm.auto import tqdm

In [3]:
ztfb, elts = load_ztf_easy_batch(batch_size=64)

In [4]:
ztfb

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,ast_uy,ast_uz,elt_ux,elt_uy,elt_uz,elt_r,elt_sec,score,is_hit,is_match
0,37606,733,b'ZTF18abtdsvv',621336711115010010,9577,58375.336713,350.405941,0.879484,0.985897,-0.146790,...,-0.144934,0.079786,0.983072,-0.161029,0.087400,2.595640,3326.835799,0.000023,False,False
1,37607,733,b'ZTF18abtdsvv',621309991115010019,9572,58375.310000,350.405990,0.879483,0.985897,-0.146789,...,-0.144804,0.079784,0.983090,-0.160937,0.087369,2.595627,3306.916479,0.000026,False,False
2,37610,733,b'ZTF18abtdsvv',614358404915010002,8240,58368.358403,350.405972,0.879528,0.985897,-0.146789,...,-0.145699,0.082020,0.987293,-0.137772,0.079197,2.599708,1897.631312,0.031018,False,False
3,37612,733,b'ZTF18abtdsvv',615324371115015021,8425,58369.324375,350.405987,0.879540,0.985897,-0.146789,...,-0.145280,0.079903,0.986748,-0.140962,0.080359,2.598268,1214.514216,0.241065,False,False
4,141833,733,b'ZTF18abwpfho',623339405515010000,9980,58377.339410,348.764546,0.189321,0.980829,-0.177448,...,-0.175586,0.081961,0.981751,-0.167709,0.089660,2.597194,2759.450676,0.000646,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90205,5584064,324582,b'ZTF18aaaknvz',1149162620015010022,96198,58903.162627,67.404504,45.304078,0.270241,0.878532,...,0.878880,0.396851,0.271240,0.872415,0.406597,1.545974,2915.647647,0.000275,False,False
90206,5584155,324582,b'ZTF20aaqjjhn',1149162620115015001,96198,58903.162627,66.164947,45.635778,0.282557,0.871180,...,0.873100,0.398528,0.271240,0.872415,0.406597,1.545974,2572.286861,0.001692,False,False
90207,5584262,324582,b'ZTF20aaqjltu',1149162621915015008,96198,58903.162627,67.072721,46.398450,0.268658,0.870797,...,0.870788,0.411746,0.271240,0.872415,0.406597,1.545974,1234.232609,0.230095,False,False
90208,5584879,324582,b'ZTF20aaqjltx',1149162621915015001,96198,58903.162627,66.993445,46.563928,0.268718,0.869478,...,0.869477,0.414487,0.271240,0.872415,0.406597,1.545974,1812.635532,0.042044,False,False


In [13]:
# Threshold and resolution
thresh_deg = 1.0
R_deg = 0.2
# Convert to cartesian distance
thresh_dist = deg2dist(thresh_deg)
R_dist = deg2dist(R_deg)
A = 1.0/R_dist**2

# Calculate mu, sigma
mu_per_obs, sigma2_per_obs = score_mean_var(A=A, thresh=thresh_dist)

num_obs = 5.69E6
mu = num_obs * mu_per_obs
sigma2 = num_obs * sigma2_per_obs
sigma = np.sqrt(sigma2)

# Report results
print(f'Per Observation:')
print(f'mu:     {mu_per_obs:5.3e}')
print(f'sigma2: {sigma2_per_obs:5.3e}')
print(f'\nFor {num_obs:.0f} observations:')
print(f'mu    = {mu:10.6f}')
print(f'sigma = {sigma:10.6f}')

Per Observation:
mu:     6.092e-06
sigma2: 3.046e-06

For 5690000 observations:
mu    =  34.665298
sigma =   4.163232


In [ ]:
is_good = (ztfb.element_id == elts.element_id[32])
is_bad = ~is_good
ztf_g = ztfb[is_good]
ztf_b = ztfb[is_bad]

In [ ]:
report_ztf_score(ztf_g)

In [ ]:
report_ztf_score(ztf_b)

In [ ]:
ztf = load_ztf_nearest_ast() 
mjd = np.unique(ztf.mjd)

In [ ]:
# element_id is the unique identifier for each orbital element considered
element_id = elts.element_id.values

# Compute mjd0 and mjd1 from mjd_unq
mjd0 = np.floor(np.min(mjd))
mjd1 = np.ceil(np.max(mjd))

# Calculate positions in this date range, sampled daily
df_ast_daily, df_earth_daily, df_sun_daily = calc_ast_data(elts=elts, mjd0=mjd0, mjd1=mjd1, element_id=element_id)

# Spline positions at ztf times
df_ast, df_earth, df_sun = spline_ast_vec_df(df_ast=df_ast_daily, df_earth=df_earth_daily, df_sun=df_sun_daily, 
                                             mjd=mjd, include_elts=False)
# Direction from palomar
df_dir = calc_ast_dir(df_ast=df_ast, df_earth=df_earth, site_name='palomar')


In [ ]:
df_dir

In [ ]:
thresh_deg = 1.0
progbar=True

In [ ]:
# Get unique element IDs
element_ids = df_dir.element_id.values
element_ids_unq = np.unique(element_ids)

# Column collections used on ztf
cols_catalog = ['ObjectID', 'CandidateID', 'TimeStampID', 'mjd']
cols_radec = ['ra', 'dec']
cols_dir = ['ux', 'uy', 'uz']
cols_elt_dir = ['elt_ux', 'elt_uy', 'elt_uz']
# Add nearest asteroid data to columns if available
cols_nearest_ast_all = ['nearest_ast_num', 'nearest_ast_dist', 
                        'ast_ra', 'ast_dec', 'ast_ux', 'ast_uy', 'ast_uz']
cols_nearest_ast = [col for col in cols_nearest_ast_all if col in ztf.columns]
# All the output columns
cols_out = cols_catalog + cols_radec + cols_dir + cols_nearest_ast

# Extract TimeStampID as (M,) array; name it row_num to emphasize that we use it to index into u_elt
row_num = ztf.TimeStampID.values

# Extract directions of the ZTF observations as an Mx3 array
u_ztf = ztf[cols_dir].values

# Threshold as Cartesian distance
thresh_dist = deg2dist(thresh_deg)    

# Dictionary of DataFrames that are close
ztf_tbl = dict()

# Iterate over distinct element IDs
iterates = tqdm(element_ids_unq) if progbar else element_ids_unq
for element_id in iterates:
    # projected directions with this element id
    mask_elt = (df_dir.element_id == element_id)
    # Directions of this candidate element at the unique time stamps
    u_elt = df_dir.loc[mask_elt, cols_dir].values
    r_elt = df_dir.loc[mask_elt, 'delta'].values
    # Difference bewteen ztf direction and this element's direction
    dist_i = np.linalg.norm(u_ztf - u_elt[row_num], axis=1)
    # Which rows are within the threshold distance?
    mask_close = (dist_i < thresh_dist)
    # Row numbers corresponding to close observations
    row_num_close = row_num[mask_close]
    # Copy this slice
    ztf_i = ztf.loc[mask_close, cols_out].copy()
    # Insert columns with the ztf_id and element_id
    ztf_i.insert(loc=0, column='ztf_id', value=ztf_i.index.values)
    ztf_i.insert(loc=1, column='element_id', value=element_id)
    # Insert columns with the predicted directions of the elements
    for col in cols_elt_dir:
        ztf_i.insert(loc=ztf_i.columns.size, column=col, value=0.0)
    # ztf_i[cols_elt_dir] = u_elt[row_num]
    # ztf_i['elt_r'] = r_elt[row_num]
    # Save it to the table (dict) of elements
    ztf_tbl[element_id] = ztf_i

In [ ]:
ztf_i

In [ ]:
u_elt.shape

In [ ]:
row_num

In [ ]:
row_num_close

In [ ]:
row_num_close.size